In [1]:
%%time
import malaya

CPU times: user 5.85 s, sys: 1.41 s, total: 7.26 s
Wall time: 11.9 s


In [2]:
string1 = 'xjdi ke, y u xsuke makan HUSEIN kt situ tmpt, i hate it. pelikle, pada'
string2 = 'i mmg xske mknn HUSEIN kampng tmpat, i love them. pelikle saye'
string3 = 'perdana menteri ke11 sgt suka makn ayam, harganya cuma rm15.50'
string4 = 'pada 10/4, kementerian mengumumkan, 1/100'
string5 = 'Husein Zolkepli dapat tempat ke-12 lumba lari hari ni'
string6 = 'Husein Zolkepli (2011 - 2019) adalah ketua kampng di kedah sekolah King Edward ke-IV'

## Load spell normalizer

In [3]:
corrector = malaya.spell.probability()
normalizer = malaya.normalize.spell(corrector)

In [4]:
print(normalizer.normalize(string1))
print(normalizer.normalize(string2))
print(normalizer.normalize(string3))
print(normalizer.normalize(string4))
print(normalizer.normalize(string5))
print(normalizer.normalize(string6))

tak jadi ke , kenapa awak tak suka makan HUSEIN kat situ tmpat , saya hate it . pelik lah , pada
saya memang tak suka makanan HUSEIN kampung tempat , saya love them . pelik lah saya
perdana menteri kesebelas sangat suka makan ayam , harganya cuma lima belas perpuluhan lima ringgit
pada sepuluh hari bulan empat , kementerian mengumumkan , satu per seratus
Husein Zolkepli dapat tempat kedua belas lumba lari hari ni
Husein Zolkepli ( dua ribu sebelas hingga dua ribu sembilan belas ) adalah ketua kampung di kedah sekolah King Edward keempat


## Normalizing rules

#### 1. Normalize title,

```python

{
    'dr': 'Doktor',
    'yb': 'Yang Berhormat',
    'hj': 'Haji',
    'ybm': 'Yang Berhormat Mulia',
    'tyt': 'Tuan Yang Terutama',
    'yab': 'Yang Berhormat',
    'ybm': 'Yang Berhormat Mulia',
    'yabhg': 'Yang Amat Berbahagia',
    'ybhg': 'Yang Berbahagia',
    'miss': 'Cik',
}

```

In [5]:
normalizer.normalize('Dr yahaya')

'Doktor yahaya'

#### 2. expand `x`

In [6]:
normalizer.normalize('xtahu')

'tak tahu'

#### 3. normalize `ke -`

In [7]:
normalizer.normalize('ke-12')

'kedua belas'

In [8]:
normalizer.normalize('ke - 12')

'kedua belas'

#### 4. normalize `ke - roman`

In [9]:
normalizer.normalize('ke-XXI')

'kedua puluh satu'

In [10]:
normalizer.normalize('ke - XXI')

'kedua puluh satu'

#### 5. normalize `NUM - NUM`

In [11]:
normalizer.normalize('2011 - 2019')

'dua ribu sebelas hingga dua ribu sembilan belas'

In [12]:
normalizer.normalize('2011.01-2019')

'dua ribu sebelas perpuluhan kosong satu hingga dua ribu sembilan belas'

#### 6. normalize `pada NUM (/ | -) NUM`

In [13]:
normalizer.normalize('pada 10/4')

'pada sepuluh hari bulan empat'

In [14]:
normalizer.normalize('PADA 10 -4')

'pada sepuluh hari bulan empat'

#### 7. normalize `NUM / NUM`

In [15]:
normalizer.normalize('10 /4')

'sepuluh per empat'

#### 8. normalize `rm NUM`

In [16]:
normalizer.normalize('RM 10.5')

'sepuluh perpuluhan lima ringgit'

#### 9. normalize `rm NUM sen`

In [17]:
normalizer.normalize('rm 10.5 sen')

'sepuluh perpuluhan lima ringgit'

#### 10. normalize `NUM sen`

In [18]:
normalizer.normalize('10.5 sen')

'sepuluh perpuluhan lima sen'

#### 11. normalize money

In [19]:
normalizer.normalize('rm10.4m')

'sepuluh perpuluhan empat juta ringgit'

In [20]:
normalizer.normalize('$10.4M')

'sepuluh perpuluhan empat juta dollar'

In [21]:
normalizer.normalize('rm10.4b')

'sepuluh perpuluhan empat billion ringgit'

#### 12. normalize cardinal

In [22]:
normalizer.normalize('123')

'seratus dua puluh tiga'

#### 13. normalize ordinal

In [23]:
normalizer.normalize('ke123')

'keseratus dua puluh tiga'